In [1]:
from sklearn.metrics import ndcg_score, dcg_score
import numpy as np

true = np.asarray([[3, 2, 1, 0, 0]])
relevance = np.asarray([[3, 2, 0, 0, 1]])

print(ndcg_score(true, relevance))

#0.980840401274087

0.980840401274087


Задание 4.5

Пусть у нас есть реальные оценки, выставленные пользователем, и предсказанные оценки:

In [2]:
r = np.asarray([[2, 4, 1, 1, 1]])
p = np.asarray([[2, 5, 2, 3, 1]])

print(ndcg_score(r, p))

0.9748317848747132


Давайте на примере рассмотрим, как можно построить систему рекомендаций на основе популярности.

Мы будем работать с набором данных Movie Lens. Он содержит идентификаторы для каждого фильма и пользователя, который его смотрел, а также оценку, которую пользователь поставил фильму. В датасете представлено 25 000 095 оценок фильмов от 162 541 пользователя со шкалой оценок от 0.5 до 5.0.

Из этого набора нам понадобится два файла:

данные о фильмах (movies);
данные о выставленных оценках (ratings).

Объединим их:

In [4]:
import pandas as pd
movies = pd.read_csv('/Users/egor/Documents/data_science_course/SKILLFACTORY/MATH&ML-14. Рекомендательные системы. Часть I/data/movie.csv')
ratings = pd.read_csv('/Users/egor/Documents/data_science_course/SKILLFACTORY/MATH&ML-14. Рекомендательные системы. Часть I/data/dst-3.0_mathml_14_5_rating.csv')
df=pd.merge(ratings, movies, how='left',on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


userId — id пользователя;
movieId — id фильма;
rating — выставленный пользователем рейтинг для фильма;
timestamp — время выставления рейтинга;
title — название фильма;
genres — жанры, к которым относится фильм.

Задание 5.1

Подсчитайте, сколько раз каждый фильм встречается в наборе данных. Отметьте среди перечисленных ниже фильмов те, что встречаются в топ-5 по популярности.

In [6]:
df['title'].value_counts().head(5)

title
Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: count, dtype: int64

Задание 5.2

Отлично, мы нашли самые востребованные фильмы. Однако если фильм посмотрело много людей, это ещё не значит, что он им понравился. Чтобы понять, как зритель на самом деле относится к фильму, нужны более чёткие данные. К счастью, в наборе данных Movie Lens есть оценки каждого из зрителей.

- Найдите средний рейтинг для каждого из фильмов.
- Найдите фильмы с наивысшим средним рейтингом.
- Введите в качестве ответа фильм, занимающий последнее место среди фильмов с наивысшим рейтингом, если предварительно отсортировать их по алфавитному порядку.
Впишите только название фильма, без кавычек и без года выхода на экран.

In [10]:
film_rating = df.groupby(by='title')['rating'].mean().sort_values(ascending=False)

In [12]:
film_rating[film_rating == 5].sort_index()

title
1971 (2014)                                       5.0
A Blank on the Map (1971)                         5.0
A Gun for Jennifer (1997)                         5.0
A Night for Dying Tigers (2010)                   5.0
Abendland (2011)                                  5.0
                                                 ... 
Welcome to Australia (1999)                       5.0
When I Walk (2013)                                5.0
Who Killed Vincent Chin? (1987)                   5.0
Year Zero: The Silent Death of Cambodia (1979)    5.0
Yonkers Joe (2008)                                5.0
Name: rating, Length: 113, dtype: float64

Задание 5.3

В двух предыдущих заданиях вы использовали два самых простых метода для создания неперсонализированных рекомендаций. Однако у них обоих есть свои недостатки: поиск наиболее часто просматриваемых фильмов не учитывает того, насколько фильм нравится аудитории, а поиск среднего рейтинга может вывести в рекомендуемые фильмы малоизвестные специфические картины с одной-двумя оценками.

Чтобы решить эти проблемы, объединим два подхода и будем искать средний рейтинг только для фильмов, которые были оценены более 50 раз.

Сколько таких фильмов?

In [29]:
view_count_all = df['title'].value_counts()
view_count = view_count_all[view_count_all > 50]
view_count.shape[0]

10472

In [30]:
new_df = df[df['title'].isin(view_count.index)]
new_df.groupby(by='title')['rating'].mean().sort_values(ascending=False)

title
Shawshank Redemption, The (1994)       4.446990
Godfather, The (1972)                  4.364732
Usual Suspects, The (1995)             4.334372
Schindler's List (1993)                4.310175
Godfather: Part II, The (1974)         4.275641
                                         ...   
Barney's Great Adventure (1998)        1.163484
Glitter (2001)                         1.124088
Bratz: The Movie (2007)                1.105556
From Justin to Kelly (2003)            0.973005
SuperBabies: Baby Geniuses 2 (2004)    0.837321
Name: rating, Length: 10472, dtype: float64